In [1]:
import os
import sys
import pandas as pd
pd.set_option('display.max_rows', None)  # Set to None to display all rows
import urllib
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime, timedelta, timezone

# Add project root to sys.path
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Verify sys.path
print("Current sys.path:", sys.path)

from src.db.base_utils import get_db_connection
from src.db.the_odds_db_utils import *

Current sys.path: ['C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv', '', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper']


c:\Users\H-100\Desktop\python\accurateshothelper\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Today's date (EST)
(datetime.now() - timedelta(hours=6)).strftime('%Y-%m-%d')

'2025-01-31'

In [3]:
# fetch_and_store_nhl_games('2025-01-30', enable_logging=True)

In [4]:
# get_nhl_events_from_db('2025-01-06', enable_logging=True)

In [5]:
get_mismatched_game_ids_with_details(enable_logging=True)


INFO:root:Comparing distinct game_ids in game_info and player_sog_odds tables.
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Game IDs only in game_info: []
INFO:root:Game IDs only in player_sog_odds: set()
INFO:root:Completed comparison of game_ids.


{'only_in_game_info': [], 'only_in_odds_table': []}

In [6]:
# process_sog_markets('76d32822c0db9bfcc89357f081134d89', '2025-01-05', enable_logging=True)

In [7]:
process_all_sog_markets('2025-01-25', enable_logging=True)

INFO:root:Processing all SOG markets for date: 2025-01-25
INFO:root:Retrieving NHL events from DB for date: 2025-01-25
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 13 events from the database for date 2025-01-25.
INFO:root:Processing game e46de5e0481c35effe3b6f1825a83be0 scheduled for 2025-01-25T18:00:00Z
INFO:root:Processing SOG markets for game: e46de5e0481c35effe3b6f1825a83be0
INFO:root:Using historical odds for date: 2025-01-25T18:00:00Z
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Inserted/Updated 108 records for game e46de5e0481c35effe3b6f1825a83be0
INFO:root:Completed processing SOG markets for game e46de5e0481c35effe3b6f1825a83be0
INFO:root:Processing game b13cdf3e397b8cfdb0f165bd137

In [8]:
def process_all_sog_markets_for_month(year: int, month: int, enable_logging: bool = True) -> None:
    """
    Runs the `process_all_sog_markets` function for each day in the specified month.

    Args:
        year (int): The year of the month to process.
        month (int): The month to process (1-12).
        enable_logging (bool, optional): Enable logging. Defaults to True.
    """
    import calendar
    from datetime import datetime

    # Get the number of days in the specified month
    num_days = calendar.monthrange(year, month)[1]

    for day in range(1, num_days + 1):
        # Format the date as 'YYYY-MM-DD'
        date_str = datetime(year, month, day).strftime('%Y-%m-%d')
        
        # Log the current date being processed
        if enable_logging:
            print(f"Processing SOG markets for date: {date_str}")
        
        # Call the existing function to process SOG markets for the current date
        process_all_sog_markets(date_str, enable_logging=enable_logging)

In [9]:
# process_all_sog_markets_for_month(year= 2024, month= 11)

In [ ]:
def process_all_sog_markets_for_date_range(start_date: str, end_date: str, enable_logging: bool = True) -> None:
    """
    Runs the `process_all_sog_markets` function for each date within the specified date range.
    
    Args:
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.
        enable_logging (bool, optional): Enable logging. Defaults to True.
    """
    from datetime import datetime, timedelta

    # Convert string dates to datetime objects
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')

    if start_dt > end_dt:
        raise ValueError("start_date must be on or before end_date")

    current_dt = start_dt
    while current_dt <= end_dt:
        date_str = current_dt.strftime('%Y-%m-%d')
        
        # Log the current date being processed
        if enable_logging:
            print(f"Processing SOG markets for date: {date_str}")
        
        # Call the existing function to process SOG markets for the current date
        process_all_sog_markets(date_str, enable_logging=enable_logging)
        
        # Move to the next day
        current_dt += timedelta(days=1)

# Example usage
# process_all_sog_markets_for_date_range(start_date='2024-04-01', end_date='2024-04-18')

In [13]:
def process_all_moneyline_markets_for_date_range(start_date: str, end_date: str, enable_logging: bool = True) -> None:
    """
    Runs the `process_all_moneyline_markets` function for each date within the specified date range.
    
    Args:
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.
        enable_logging (bool, optional): Enable logging. Defaults to True.
    """
    from datetime import datetime, timedelta

    # Convert string dates to datetime objects
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')

    if start_dt > end_dt:
        raise ValueError("start_date must be on or before end_date")

    current_dt = start_dt
    while current_dt <= end_dt:
        date_str = current_dt.strftime('%Y-%m-%d')
        
        # Log the current date being processed
        if enable_logging:
            print(f"Processing moneyline markets for date: {date_str}")
        
        # Call the existing function to process moneyline markets for the current date
        process_all_moneyline_markets(query_date=date_str, enable_logging=enable_logging)
        
        # Move to the next day
        current_dt += timedelta(days=1)

# Example usage
process_all_moneyline_markets_for_date_range(start_date='2024-03-01', end_date='2024-03-31')

INFO:root:Processing all moneyline markets for date: 2024-03-01
INFO:root:Retrieving NHL events from DB for date: 2024-03-01
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 0 events from the database for date 2024-03-01.
INFO:root:No games found for date 2024-03-01, attempting to fetch from API
INFO:root:Fetching and storing NHL events for date: 2024-03-01
INFO:root:Using historical API for date: 2024-03-01


Processing moneyline markets for date: 2024-03-01


INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Inserted/Updated 15 records into game_info table.
INFO:root:Completed fetching and storing events.
INFO:root:Retrieving NHL events from DB for date: 2024-03-01
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 3 events from the database for date 2024-03-01.
INFO:root:Processing game 366a884213a807c78c1234f0a6858687 scheduled for 2024-03-02T00:00:00Z
INFO:root:Processing moneyline markets for game: 366a884213a807c78c1234f0a6858687
INFO:root:Using historical odds for date: 2024-03-02T00:00:00Z
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established dat

Processing moneyline markets for date: 2024-03-02


INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Inserted/Updated 16 records into game_info table.
INFO:root:Completed fetching and storing events.
INFO:root:Retrieving NHL events from DB for date: 2024-03-02
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 13 events from the database for date 2024-03-02.
INFO:root:Processing game 5b94e21ca4aea2498db058958781de9a scheduled for 2024-03-02T17:30:00Z
INFO:root:Processing moneyline markets for game: 5b94e21ca4aea2498db058958781de9a
INFO:root:Using historical odds for date: 2024-03-02T17:30:00Z
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established da

...

Processing moneyline markets for date: 2024-03-27


INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Inserted/Updated 14 records into game_info table.
INFO:root:Completed fetching and storing events.
INFO:root:Retrieving NHL events from DB for date: 2024-03-27
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 2 events from the database for date 2024-03-27.
INFO:root:Processing game 52d03d9df55ce489443f96eba3cc05d3 scheduled for 2024-03-27T23:00:00Z
INFO:root:Processing moneyline markets for game: 52d03d9df55ce489443f96eba3cc05d3
INFO:root:Using historical odds for date: 2024-03-27T23:00:00Z
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established dat

Processing moneyline markets for date: 2024-03-28


INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Inserted/Updated 16 records into game_info table.
INFO:root:Completed fetching and storing events.
INFO:root:Retrieving NHL events from DB for date: 2024-03-28
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 14 events from the database for date 2024-03-28.
INFO:root:Processing game 96a7cb61d0cfa69a2860e0522a75f86c scheduled for 2024-03-28T23:00:00Z
INFO:root:Processing moneyline markets for game: 96a7cb61d0cfa69a2860e0522a75f86c
INFO:root:Using historical odds for date: 2024-03-28T23:00:00Z
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established da

AttributeError: 'NoneType' object has no attribute 'get'